<a href="https://colab.research.google.com/github/acafiero/sports-project2/blob/main/Cafiero_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

 ## *Dataset 1*
 1) github / Five Thirty Eight

 2) statistical data for modern Toronto Raptors players

 3) Wins Above Replacement in the regular season

 4) Regression

 5) 21 features

 6) 4078 rows

 7) There are multiple entries for the same player but from differing years, I would imagine I might need to either do averages for all the years these players played or pick a specific year to focus on

In [8]:
df = pd.read_csv('/content/modern_RAPTOR_by_player.csv')

In [9]:
df.head()

,player_name,player_id,season,poss,mp,raptor_box_offense,raptor_box_defense,raptor_box_total,raptor_onoff_offense,raptor_onoff_defense,...,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alex Abrines,abrinal01,2017,2387,1135,0.745505,-0.372938,0.372567,-0.418553,-3.857011,...,0.543421,-1.144832,-0.601411,1.249008,1.447708,-0.198700,0.077102,-1.038677,-0.961575,0.326413
1,Alex Abrines,abrinal01,2018,2546,1244,0.317549,-1.725325,-1.407776,-1.291727,-0.049694,...,-0.020826,-1.502642,-1.523468,0.777304,0.465912,0.311392,-0.174621,-1.112625,-1.287247,-0.456141
2,Alex Abrines,abrinal01,2019,1279,588,-3.215683,1.078399,-2.137285,-6.158856,4.901168,...,-4.040157,1.885618,-2.154538,0.178167,0.178167,0.000000,-4.577678,1.543282,-3.034396,-0.268013
3,Precious Achiuwa,achiupr01,2021,1581,749,-4.122966,1.359278,-2.763688,-4.050779,-0.919712,...,-4.347596,0.954821,-3.392775,-0.246055,-0.246776,0.000721,-3.817713,0.474828,-3.342885,0.329157
4,Quincy Acy,acyqu01,2014,1716,847,-1.716079,0.133115,-1.582964,-0.324811,-1.661495,...,-1.565257,-0.216477,-1.781734,0.415918,0.415918,0.000000,-1.464417,-0.223754,-1.688171,-0.554898


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4079 entries, 0 to 4078
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   player_name           4079 non-null   object 
 1   player_id             4079 non-null   object 
 2   season                4079 non-null   int64  
 3   poss                  4079 non-null   int64  
 4   mp                    4079 non-null   int64  
 5   raptor_box_offense    4078 non-null   float64
 6   raptor_box_defense    4078 non-null   float64
 7   raptor_box_total      4078 non-null   float64
 8   raptor_onoff_offense  4078 non-null   float64
 9   raptor_onoff_defense  4078 non-null   float64
 10  raptor_onoff_total    4078 non-null   float64
 11  raptor_offense        4079 non-null   float64
 12  raptor_defense        4079 non-null   float64
 13  raptor_total          4079 non-null   float64
 14  war_total             4079 non-null   float64
 15  war_reg_season       

## *Dataset 2*

1) github / Five Thirty Eight

2) brief overview of information about past NCAA Tournaments, who was the projected favorite in matchups, who was a projected underdog, etc

3) who was more favored in their probability

4) classification

5) 6 features

6) 252 rows of data

7) I would imagine there will probably be some strange projections that statistically made sense but realistically improbable. Otherwise it is a pretty straightforward dataset.

In [12]:
df2 = pd.read_csv('/content/ncaa tourney raw.csv')

In [13]:
df2.head()

,year,round,favorite,underdog,favorite_probability,favorite_win_flag
0,2014,2,Texas,Arizona State,0.501,1
1,2013,2,Illinois,Colorado,0.504,1
2,2013,1,James Madison,Long Island,0.506,1
3,2011,2,Cincinnati,Missouri,0.509,1
4,2012,3,Cincinnati,Florida State,0.509,1


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  253 non-null    int64  
 1   round                 253 non-null    int64  
 2   favorite              253 non-null    object 
 3   underdog              253 non-null    object 
 4   favorite_probability  253 non-null    float64
 5   favorite_win_flag     253 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 12.0+ KB
